In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
#from nltk.corpus import stopwords
from nltk import word_tokenize
#STOPWORDS = set(stopwords.words('english'))
from bs4 import BeautifulSoup
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

Using TensorFlow backend.


In [2]:
import re
import unicodedata

In [45]:
from imblearn.pipeline import Pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE,RandomOverSampler
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import balanced_accuracy_score

In [4]:
df=pd.read_csv('data/train.csv')

In [5]:
temp1=df[df['label_quality']=='unreliable']['category'].tolist()
temp2=df[df['label_quality']=='reliable']['category'].tolist()

cat_not_in_reliable=list(set(temp1) - set(temp2))
cat_count=df[(df['label_quality']=='reliable')|(df['category'].isin(cat_not_in_reliable))]['category'].value_counts()
cat_low_count=cat_count[cat_count<4000].index.tolist()
df=df[(df['label_quality']=='reliable')|(df['category'].isin(cat_not_in_reliable))|(df['category'].isin(cat_low_count))]
all_cat=df['category'].value_counts()
all_cat=all_cat[all_cat<=400].index.tolist()
df_spanish_temp        = df[df['language']=='spanish']
counts_spanish         = df_spanish_temp['category'].value_counts()
low_spanish            =counts_spanish[counts_spanish<200]
df_spanish=df[(df['language']=='spanish')|\
              (~df['category'].isin(counts_spanish.index))|\
              (df['category'].isin(all_cat))|\
              (df['category'].isin(low_spanish.index))]

sampling_spanish={}
counts_spanish  = df_spanish['category'].value_counts()
for cat in counts_spanish.index:
    if counts_spanish.loc[cat]>2200:
        sampling_spanish[cat]=2200
    else:
        sampling_spanish[cat]=counts_spanish.loc[cat]

X_spanish = df_spanish['title']
y_spanish = df_spanish['category']

rus_spanish = RandomUnderSampler(sampling_strategy=sampling_spanish,random_state=42)
X_res_spanish, y_res_spanish = rus_spanish.fit_resample(X_spanish.values.reshape(-1, 1), y_spanish)

#SPANISH
X_res_spanish=pd.DataFrame(X_res_spanish)
X_res_spanish.columns=['title']
X_res_spanish['title']=X_res_spanish['title'].astype('str')
y_res_spanish=pd.Series(y_res_spanish)

In [6]:
def clean_text(text) :
    text=unicodedata.normalize('NFKD', str(text)).encode('ascii', errors='ignore').decode('utf-8')\
    .lower().replace(r'\\n','').replace(r'-',' ').replace(r'.','').strip()
    text = re.sub("[^a-zA-Z]", " ", str(text))
    #return " ".join(text.split())
    return " ".join([w for w in text.split() if len(w)>1])

In [7]:
X_res_spanish['title']=X_res_spanish['title'].apply(clean_text)

In [57]:
y_res_spanish.value_counts()

POWER_GRINDERS                          2200
PLANT_POTS_AND_PLANTERS                 2200
PANTS                                   2200
TRANSISTORS                             2200
BATHROOM_VANITIES                       2200
                                        ... 
CONSTRUCTION_LIME_BAGS                   206
COLD_FOOD_AND_DRINK_VENDING_MACHINES     162
PAINTBALL_SMOKE_GRENADES                 154
COMMERCIAL_POPCORN_MACHINES              141
HAMBURGER_FORMERS                        109
Length: 1588, dtype: int64

In [67]:
# RandomOverSampler

sampling_ros_spanish={}
counts_spanish  = y_res_spanish.value_counts()
for cat in counts_spanish.index:
    if counts_spanish.loc[cat]>1800:
        sampling_ros_spanish[cat]=counts_spanish.loc[cat]
    else:
        sampling_ros_spanish[cat]=int(counts_spanish.loc[cat]+counts_spanish.loc[cat]*.1)

In [68]:
ros = RandomOverSampler(sampling_strategy=sampling_ros_spanish,random_state=42)
X_ros_spanish, y_ros_spanish = ros.fit_sample(X_res_spanish.values.reshape(-1, 1), y_res_spanish)

In [69]:
pd.Series(y_ros_spanish).value_counts()

POWER_GRINDERS                          2200
PLANT_POTS_AND_PLANTERS                 2200
PANTS                                   2200
TRANSISTORS                             2200
BATHROOM_VANITIES                       2200
                                        ... 
CONSTRUCTION_LIME_BAGS                   226
COLD_FOOD_AND_DRINK_VENDING_MACHINES     178
PAINTBALL_SMOKE_GRENADES                 169
COMMERCIAL_POPCORN_MACHINES              155
HAMBURGER_FORMERS                        119
Length: 1588, dtype: int64

In [70]:
#SPANISH
X_ros_spanish=pd.DataFrame(X_ros_spanish)
X_ros_spanish.columns=['title']
X_ros_spanish['title']=X_ros_spanish['title'].astype('str')
y_ros_spanish=pd.Series(y_ros_spanish)

In [72]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each title.
MAX_SEQUENCE_LENGTH = 40
# This is fixed.
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(X_ros_spanish['title'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 295737 unique tokens.


In [73]:
X = tokenizer.texts_to_sequences(X_ros_spanish['title'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (2996106, 40)


In [74]:
y = pd.get_dummies(y_ros_spanish).values
print('Shape of label tensor:', y.shape)

Shape of label tensor: (2996106, 1588)


In [75]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.5, random_state = 42,)# stratify=y)
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(1498053, 40) (1498053, 1588)
(1498053, 40) (1498053, 1588)


In [76]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1588, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy']) # accuracy
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 100)           5000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 40, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1588)              160388    
Total params: 5,240,788
Trainable params: 5,240,788
Non-trainable params: 0
_________________________________________________________________
None


In [77]:
epochs = 7
batch_size = 64
history = model.fit(X_train, y_train,
                    epochs=epochs, batch_size=batch_size,
                    validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

e:\anaconda\envs\keras\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning:

Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.



Train on 1348247 samples, validate on 149806 samples
Epoch 1/7
1348247/1348247 [==============================] - 2235s 2ms/step - loss: 2.1247 - categorical_accuracy: 0.6347 - val_loss: 0.9516 - val_categorical_accuracy: 0.8184
Epoch 2/7
1348247/1348247 [==============================] - 2252s 2ms/step - loss: 0.9435 - categorical_accuracy: 0.8151 - val_loss: 0.8420 - val_categorical_accuracy: 0.8366
Epoch 3/7
1348247/1348247 [==============================] - 2136s 2ms/step - loss: 0.8219 - categorical_accuracy: 0.8337 - val_loss: 0.8108 - val_categorical_accuracy: 0.8435
Epoch 4/7
1348247/1348247 [==============================] - 2142s 2ms/step - loss: 0.7625 - categorical_accuracy: 0.8428 - val_loss: 0.7966 - val_categorical_accuracy: 0.8464
Epoch 5/7
1348247/1348247 [==============================] - 2140s 2ms/step - loss: 0.7243 - categorical_accuracy: 0.8489 - val_loss: 0.7870 - val_categorical_accuracy: 0.8482
Epoch 6/7
1348247/1348247 [==============================] - 2135s 

In [78]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

1498053/1498053 [==============================] - 304s 203us/step
Test set
  Loss: 0.788
  Accuracy: 0.849


In [ ]:
0.846

In [79]:
labels = pd.get_dummies(y_res_spanish).columns

In [93]:
title = 'Te Rojo En Hebras   X 1 Kg Nacional Imperdible'
title_clean = clean_text(title)
seq = tokenizer.texts_to_sequences([title_clean])
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)
print(labels[np.argmax(pred)])

TEA


In [80]:
model.save('models/LSTM_spanish.h5')

# SUBMIT

In [81]:
df_test = pd.read_csv('data/test.csv')

In [82]:
df_pred_spanish=df_test[df_test['language']=='spanish']
df_pred_spanish['title']=df_pred_spanish['title'].apply(clean_text)

In [83]:
def prediction(title):
    seq = tokenizer.texts_to_sequences([title])
    padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
    pred = model.predict(padded)
    return labels[np.argmax(pred)]

In [84]:
df_pred_spanish['category'] = df_pred_spanish['title'].apply(prediction)

In [85]:
df_pred_spanish.head()

,id,title,language,category
9,9,disco rigido externo western digital elements ...,spanish,HARD_DRIVES_AND_SSDS
10,10,picadora de carne fineschi legitima,spanish,MEAT_GRINDERS
14,14,set barreta automotor bremen unid cm,spanish,CROWBARS
15,15,miel organica gr sin tacc,spanish,HONEY
19,19,bandeja giradiscos omnitronic bd,spanish,TURNTABLES


In [86]:
df_pred_spanish.to_csv('submit/submit_lstm_spanish.csv',index=False)